In [0]:
geoLocationSinkLayerName = "bronze"
geoLocationSinkStorageAccountName = "adlsudadalakehousedev"
geoLocationSinkFolderName = "geo-location"

geoLocationSinkFolderPath = f"abfss://{geoLocationSinkLayerName}@{geoLocationSinkStorageAccountName}.dfs.core.windows.net/{geoLocationSinkFolderName}"

geoLocationDF = (spark 
                 .read
                 .json(geoLocationSinkFolderPath))

In [0]:
display(geoLocationDF)


In [0]:
import pandas as pd
from pyspark.sql.functions import col


geoLocationSparkDF =(spark
                     .createDataFrame([finalelement for element in geoLocationDF.select("results").collect() for finalelement in element["results"]])
                     .select(col("admin1").alias("stateName"),
                     col("admin2").alias("districtName"),
                     col("country").alias("countryName"),
                     col("latitude"),
                     col("longitude"),
                     col("name").alias("marketName"),
                     col("population")
                     ))

In [0]:
display(geoLocationSparkDF)

In [0]:
(geoLocationSparkDF
 .write
 .mode("overwrite")
 .saveAsTable("pricing_analytics.silver.geo_location_silver"))

In [0]:
dailyPricingDF = spark.sql( "select * from pricing_analytics.silver.daily_pricing_silver")

In [0]:
dailyPricingLocationDF =dailyPricingDF.join(geoLocationSparkDF,((dailyPricingDF.MARKET_NAME == geoLocationSparkDF.marketName) & (dailyPricingDF.STATE_NAME == geoLocationSparkDF.stateName)), "inner")

In [0]:
display(dailyPricingLocationDF)